# Podcast Episode Erstellen

### imports

In [10]:
import sys
sys.path.append(".")
from audio_transcription.faster_whisper_word_level import transcribe
from Episodes_Downloader.episodes_downloader import get_names_and_urls_all_episodes, download_and_save_mp3_in_dir
import os
import pandas as pd
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df,db_save_df
from segment_ranking.rank_segments import get_most_similar_documents_Llama2
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from tqdm import tqdm

### Episoden URLs laden

In [7]:
titles, audio_urls = get_names_and_urls_all_episodes()
print(titles)

['Selbstliebe - Wie kann man mit sich befreundet sein', 'Philosophie der Schönheit - Was den Menschen "schön" macht', 'Manufakturen - Schritt in eine neue Arbeitswelt', 'Arbeitszeit - Von der Stechuhr zum Coworking Space ', 'Ennio Morricone - Meister der Filmmusik', 'Earl Hamner, Erfinder der Waltons - Sehnsucht nach dem Landleben', 'Warum treiben Menschen Sport? - Eine Erfolgsgeschichte', 'Ernährungstheorien - Fakten oder Mythen?', 'Die Geschichte der Stadt - Urbanes Leben als Motor der Gesellschaft', 'Das Fenster - Geschichte mit Durchblick', 'Tiefenzeit - Vergangenheit und Zukunft wiederentdecken', 'Gegenwart - Ein rätselhaftes Phänomen', 'Vergebung als Chance - Ein Prozess der inneren Aussöhnung', 'Harze - Das Blut der Bäume', 'Runzelige Rinden und bunte Borken - Die Haut der Bäume', 'Tiere im Winter – Alles Natur', 'Eichhörnchen - Kletternde Kobolde', 'Der Eiffelturm - Ikone der Moderne', 'Beruf: Hausfrau - Die Geschichte einer Arbeiterin', 'Mobiles Arbeiten - Welche Fallstricke l

In [9]:
print(len(titles))

2252 2252


In [10]:
df = db_get_df("transcripts")
df.dtypes

filename         object
download_url     object
audio_binary     object
segment_count     int64
dtype: object

In [17]:
df = pd.DataFrame({"title": titles, "download_url": audio_urls, "transcript": None})
db_save_df(df, "transcripts_all")

### Alle mp3s herunterladen

In [11]:
os.listdir("/nfs/scratch/students/neumannvi84434/Podcast_Episoden/")


['Beruf: Hausfrau - Die Geschichte einer Arbeiterin',
 'Die Reliquienräuber von Bari - Nikolaus in Geschichte und Legende',
 'Die Benin-Bronzen - Faszinierende Kunst und koloniales Unrecht',
 'Steinalte Pilze - Zeugen der ältesten Ökosysteme',
 'Runzelige Rinden und bunte Borken - Die Haut der Bäume',
 'Unsere Luft - Was in ihr steckt',
 'Chassidismus - Die Welt der frommen Mystiker',
 'Finsternis und Kreativität - Eine unterschätzte Perspektive',
 'Was macht uns aus? - Der feine Unterschied zwischen Mensch und Tier',
 'Arbeitszeit - Von der Stechuhr zum Coworking Space ',
 'Petzen oder wegschauen? Umgang mit unliebsamen Wahrheiten',
 'Von der Müdigkeit - Wenn der Körper erschöpft ist',
 'Otfried Preußler - Und sein Jugendroman Krabat',
 'Kriegsenkel - Das besondere Erbe dieser Generation',
 'Der Permafrost taut auf - Folgen für Mensch und Umwelt',
 'Richard Feynman - Physikgenie mit Charisma',
 'Überfischung - Wenn das Meer leer wird',
 'Warum treiben Menschen Sport? - Eine Erfolgsges

In [15]:
df = db_get_df("transcripts_all")
path = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"
for i, row in tqdm(df.iterrows()):
    title = row["title"].replace("/","")
    if title in os.listdir("/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"):
        continue
    download_and_save_mp3_in_dir(row["download_url"], path, title)


219it [00:00, 2182.78it/s]

Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3
Downloaded MP3


219it [00:10, 2182.78it/s]

Downloaded MP3


254it [00:11, 16.79it/s]  

...

1495it [20:03,  1.13it/s]

Downloaded MP3


1496it [20:04,  1.24it/s]

Failed to download MP3. Status code: 404


TypeError: a bytes-like object is required, not 'NoneType'

In [7]:
df.loc[179, "download_url"]

'https://media.neuland.br.de/file/1828341/c/feed/hunde-und-ihr-geruchssinn-die-supernasen-im-tierreich.mp3'

### Transkription auf Word level Ebene

In [5]:
df = transcribe("/Users/br/Projects/Bachelorarbeit/data/Episode_audio_files/jonathan swift gullivers reisen 2.mp3")
# db_save_df(df,"transcript_word_level")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Estimating duration from bitrate, this may be inaccurate


[4.52s -> 5.42s]  ARD.
[5.92s -> 6.12s]  Radio
[6.12s -> 6.58s]  Wissen.
[7.44s -> 7.50s]  Die
[7.50s -> 7.96s]  ganze
[7.96s -> 8.30s]  Welt
[8.30s -> 8.62s]  des
[8.62s -> 9.40s]  Wissens.
[10.36s -> 10.42s]  Ein
[10.42s -> 10.96s]  Podcast
[10.96s -> 11.30s]  von
[11.30s -> 11.74s]  bayern
[11.74s -> 12.20s]  2
[12.20s -> 12.50s]  in
[12.50s -> 12.80s]  der
[12.80s -> 13.40s]  ARD
[13.40s -> 13.92s] -Audiothek.
[16.52s -> 17.24s]  Lilliput,
[17.66s -> 17.88s]  Anfang
[17.88s -> 18.36s]  November
[18.36s -> 19.84s]  1699.
[21.00s -> 21.26s]  Das
[21.26s -> 21.56s]  Land
[21.56s -> 21.92s]  ist
[21.92s -> 22.24s]  in
[22.24s -> 22.86s]  Aufruhr.
[23.32s -> 23.64s]  Am
[23.64s -> 24.02s]  Strand
[24.02s -> 24.38s]  liegt
[24.38s -> 24.70s]  ein
[24.70s -> 25.22s]  Monster.
[25.94s -> 26.10s]  Ein
[26.10s -> 26.44s]  Mensch
[26.44s -> 26.74s]  wie
[26.74s -> 27.08s]  ein
[27.08s -> 27.56s]  Berg,
[28.18s -> 28.44s]  ein
[28.44s -> 29.54s]  Riese.
[29.92s -> 30.26s]  Er
[30.26s -> 30.84s

In [6]:
db_save_df(df, "transcript_gulliver_word_level")

### Satzbildung mit spacy

In [ ]:
import spacy

In [7]:
df = db_get_df("transcript_gulliver_word_level")
df.dtypes

filename     object
index         int64
word         object
start       float64
end         float64
dtype: object

Zunächst erstmal mit Whispers Punkten

In [14]:
print(f'|{df["word"][2]}|')

| Wissen.|


In [15]:
sentences = []
one_sentence = []
for word in df["word"]:
    one_sentence.append(word.strip())
    if "." in word:
        sentences.append(" ".join(one_sentence))
        one_sentence = []
print(sentences)

['ARD.', 'Radio Wissen.', 'Die ganze Welt des Wissens.', 'Ein Podcast von bayern 2 in der ARD -Audiothek.', 'Lilliput, Anfang November 1699.', 'Das Land ist in Aufruhr.', 'Am Strand liegt ein Monster.', 'Ein Mensch wie ein Berg, ein Riese.', 'Er schläft.', 'Niemand weiß, woher er kommt.', 'Die Lilliputaner schaffen Seile herbei, Flöcke und Leitern.', 'Sie binden ihn fest, am ganzen Leib.', 'Jede Haarsträhne wird einzeln am Boden fixiert.', 'Da schlägt der Riese die Augen auf.', 'Nach einer Weile spürte ich, wie etwas Lebendiges sich an meinem linken Bein bewegte, vorsichtig über meine Brust hoch kroch und fast bis an mein Kinn herankam.', 'Als ich soweit ich konnte meine Augen nach unten senkte, stellte ich fest, dass es ein menschliches Wesen von weniger als 6 Zoll Höhe war, mit Pfeil und Bogen in den Händen und einem Köcher auf dem Rücken.', 'Unterdessen merkte ich, wie wenigstens 40 weitere der gleichen Art, so vermutete ich jedenfalls, dem ersten folgten.', 'Ich war aufs Äußerste v

### Demonstration

In [ ]:
df = db_get_df(table="transcripts")

In [ ]:
userInputText = "Auf dem l Feiern und trinken"
userInputTime = 7
best_fitting = get_most_similar_documents_Llama2(userInputText, userInputTime)
print(best_fitting["segment_text"].to_markdown())

produce_snippets()
produce_audio()


In [ ]:
IPython.display.Audio("concatenated_audio.mp3")